In [7]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Flatten,Dropout,BatchNormalization,Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils
import pandas as pd
import tensorflow as tf

In [8]:
seed = 19

In [9]:
heart = pd.read_csv('heart_2020_cleaned.csv')

In [10]:
heart.tail()

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
319790,27.41,Yes,No,No,7,0,Yes,Male,Hispanic,Yes,No,Fair,6,Yes,No,No,Yes
319791,29.84,Yes,No,No,0,0,No,Male,Hispanic,No,Yes,Very good,5,Yes,No,No,No
319792,24.24,No,No,No,0,0,No,Female,Hispanic,No,Yes,Good,6,No,No,No,No
319793,32.81,No,No,No,0,0,No,Female,Hispanic,No,No,Good,12,No,No,No,No
319794,46.56,No,No,No,0,0,No,Female,Hispanic,No,Yes,Good,8,No,No,No,No


In [11]:
heart['Smoking'] = heart['Smoking'].map({'No' : 0, 'Yes' : 1})
heart['AlcoholDrinking'] = heart['AlcoholDrinking'].map({'No' : 0, 'Yes' : 1})
heart['Stroke'] = heart['Stroke'].map({'No' : 0, 'Yes' : 1})
heart['DiffWalking'] = heart['DiffWalking'].map({'No' : 0, 'Yes' : 1})
heart['Sex'] = heart['Sex'].map({'Female' : 0, 'Male' : 1})
heart['Race'] = heart['Race'].map({'White' : 0, 'Black' : 1})
heart['Diabetic'] = heart['Diabetic'].map({'No' : 0, 'Yes' : 1 , 'No, borderline diabetes' : 2})
heart['PhysicalActivity'] = heart['PhysicalActivity'].map({'No' : 0, 'Yes' : 1})
heart['GenHealth'] = heart['GenHealth'].map({'Poor' : 0, 'Fair' : 1 , 'Good' : 2 , 'Very good' : 3})
heart['Asthma'] = heart['Asthma'].map({'No' : 0, 'Yes' : 1})
heart['KidneyDisease'] = heart['KidneyDisease'].map({'No' : 0, 'Yes' : 1})
heart['SkinCancer'] = heart['SkinCancer'].map({'No' : 0, 'Yes' : 1})
heart['HeartDisease'] = heart['HeartDisease'].map({'No' : 0, 'Yes' : 1})

In [12]:
heart.head()

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
0,16.60,1,0,0,3,30,0,0,0.0,1.0,1,3.0,5,1,0,1,0
1,20.34,0,0,1,0,0,0,0,0.0,0.0,1,3.0,7,0,0,0,0
2,26.58,1,0,0,20,30,0,1,0.0,1.0,1,1.0,8,1,0,0,0
3,24.21,0,0,0,0,0,0,0,0.0,0.0,0,2.0,6,0,0,1,0
4,23.71,0,0,0,28,0,1,0,0.0,0.0,1,3.0,8,0,0,0,0


In [13]:
x = heart.iloc[:,0:16].values
x

array([[16.6 ,  1.  ,  0.  , ...,  1.  ,  0.  ,  1.  ],
       [20.34,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [26.58,  1.  ,  0.  , ...,  1.  ,  0.  ,  0.  ],
       ...,
       [24.24,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [32.81,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [46.56,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [14]:
y = heart.iloc[:,-1].values
y

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [16]:
x_train[0:5]

array([[29.29,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  1.  ,
         0.  ,  1.  ,  2.  ,  6.  ,  0.  ,  0.  ,  0.  ],
       [24.51,  1.  ,  0.  ,  0.  , 30.  ,  2.  ,  0.  ,  0.  ,  0.  ,
         1.  ,  0.  ,  2.  ,  8.  ,  0.  ,  0.  ,  1.  ],
       [24.48,  0.  ,  0.  ,  0.  , 13.  , 15.  ,  0.  ,  1.  ,  1.  ,
         0.  ,  0.  ,  1.  ,  3.  ,  0.  ,  1.  ,  0.  ],
       [19.77,  1.  ,  0.  ,  0.  ,  0.  ,  3.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  3.  ,  8.  ,  0.  ,  0.  ,  1.  ],
       [26.58,  0.  ,  0.  ,  0.  ,  0.  ,  3.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  3.  ,  7.  ,  0.  ,  0.  ,  0.  ]])

In [17]:
x_test[0:5]

array([[37.76,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  2.  ,  6.  ,  0.  ,  0.  ,  0.  ],
       [37.31,  0.  ,  0.  ,  0.  ,  3.  ,  0.  ,  0.  ,  0.  ,  1.  ,
         0.  ,  0.  ,  3.  ,  6.  ,  1.  ,  0.  ,  0.  ],
       [23.83,  0.  ,  0.  ,  0.  , 10.  , 23.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  3.  ,  6.  ,  1.  ,  0.  ,  0.  ],
       [24.19,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,   nan,
         1.  ,  1.  ,  1.  ,  6.  ,  0.  ,  0.  ,  0.  ],
       [27.32,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  0.  ,  2.  ,  7.  ,  0.  ,  0.  ,  0.  ]])

In [18]:
y_train[0:5]

array([0, 0, 0, 0, 0])

In [19]:
y_test[0:5]

array([0, 0, 0, 0, 0])

In [20]:
len(x_test)


63959

In [21]:
len(x_train)

255836

In [22]:
x_train.shape

(255836, 16)

In [23]:
from tensorflow.keras.utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

In [33]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(100, activation='relu', input_shape=(16,)))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

In [34]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               1700      
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 200)               20200     
                                                                 
 dense_7 (Dense)             (None, 2)                 402       
                                                                 
Total params: 32,402
Trainable params: 32,402
Non-trainable params: 0
_________________________________________________________________


In [36]:
class CustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(epoch % 5 ==0):
      print(" Finished epoch #",epoch)
      print('Accuracy resched train is',logs.get('accuracy'))
      print('Accuracy reached val is',logs.get('val_accuracy'))
      print()
      return

In [37]:
cc=CustomCallback()


In [38]:
history = model.fit(x_train, 
                    one_hot_train_labels, 
                    epochs=50, 
                    batch_size=256,
                    verbose=0,
                    validation_data=(x_test,one_hot_test_labels),
                    callbacks=[cc])

 Finished epoch # 0
Accuracy resched train is 0.9150432348251343
Accuracy reached val is 0.9116465449333191

 Finished epoch # 5
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished epoch # 10
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished epoch # 15
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished epoch # 20
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished epoch # 25
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished epoch # 30
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished epoch # 35
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished epoch # 40
Accuracy resched train is 0.9150940179824829
Accuracy reached val is 0.9116465449333191

 Finished ep

In [40]:
test_loss, test_acc = model.evaluate(x_test, one_hot_test_labels)

print('test_acc:', test_acc)
print('test_loss:', test_loss)

1999/1999 [==============================] - 3s 1ms/step - loss: nan - accuracy: 0.9116
test_acc: 0.9116465449333191
test_loss: nan
